In [ ]:
import sqlite3
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import torch

In [37]:
df = pd.read_csv("McD_Reviews_Sentiment_Results.csv")
df

,Review,sentiment,reply
0,"Alhamdulillah.. the best al quran application,...",Positive,BarakAllahu feekum for your kind words and sup...
1,the best quraan app you can find. it literally...,Positive,BarakAllahu feekum for your kind words and sup...
2,Very helpful,Positive,BarakAllahu feekum for your kind words and sup...
3,The recent edition does explain the source of ...,Negative,Assalamu 'alaykum dear user. We would love to ...
4,May Allah reward you richly rewarded,Positive,BarakAllahu feekum for your kind words and sup...
...,...,...,...
70455,\tVery good app,Positive,BarakAllahu feekum for your kind words and sup...
70456,"\tThank God, this application is really good!!...",Positive,BarakAllahu feekum for your kind words and sup...
70457,"\tMashallah, may God bless you",Positive,BarakAllahu feekum for your kind words and sup...
70458,\tThank God this application helps👍,Positive,BarakAllahu feekum for your kind words and sup...


In [38]:
# Load sentence transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")
# Move model to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

In [39]:
def create_faiss_index(df):
    """Creates FAISS index for fast similarity search."""
    embeddings = model.encode(df["Review"].tolist(), convert_to_numpy=True)
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    return index, embeddings

# Build FAISS index
index, embeddings = create_faiss_index(df)

In [40]:
def retrieve_reply(query, index, df, top_k=5):
    # Determine device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # Compute the query embedding on the same device
    query_embedding = model.encode([query], convert_to_numpy=True, device=device)
    
    # Search the index for the top_k closest matches
    distances, indices = index.search(query_embedding, top_k)
    
    # Gather replies and their distances
    results = []
    for dist, idx in zip(distances[0], indices[0]):
        row = df.iloc[idx]
        results.append({
            "reply": row["reply"],
            "distance": float(dist)
        })
    return results

In [41]:
query = "Very Good App"
top_matches = retrieve_reply(query, index, df, top_k=5)
for i, match in enumerate(top_matches, 1):
    print(f"{i}. Distance: {match['distance']:.4f} → Reply: {match['reply']}")

1. Distance: 0.0000 → Reply: BarakAllahu feekum for your kind words and support. Pray for us so that Allah may accept our efforts. Visit gtaf.org to check our other apps and share with others in sha Allah!
2. Distance: 0.0000 → Reply: BarakAllahu feekum for your kind words and support. Pray for us so that Allah may accept our efforts. Visit gtaf.org to check our other apps and share with others in sha Allah!
3. Distance: 0.0000 → Reply: BarakAllahu feekum for your kind words and support. Pray for us so that Allah may accept our efforts. Visit gtaf.org to check our other apps and share with others in sha Allah!
4. Distance: 0.0000 → Reply: BarakAllahu feekum for your kind words and support. Pray for us so that Allah may accept our efforts. Visit gtaf.org to check our other apps and share with others in sha Allah!
5. Distance: 0.0000 → Reply: BarakAllahu feekum for your kind words and support. Pray for us so that Allah may accept our efforts. Visit gtaf.org to check our other apps and sh

In [ ]:
query = "Very good app for Quran.But it would be better if there had a form of Fikah"
top_matches = retrieve_reply(query, index, df, top_k=5)
for i, match in enumerate(top_matches, 1):
    print(f"{i}. Distance: {match['distance']:.4f} → Reply: {match['reply']}")

1. Distance: 0.3443 → Reply: JazakAllahu khayran for your review! If you face any issues or have feature requests, please provide your feedback on feedback.gtaf.org.
2. Distance: 0.3443 → Reply: JazakAllahu khayran for your review! If you face any issues or have feature requests, please provide your feedback on feedback.gtaf.org.
3. Distance: 0.3443 → Reply: JazakAllahu khayran for your review! If you face any issues or have feature requests, please provide your feedback on feedback.gtaf.org.
4. Distance: 0.3807 → Reply: BarakAllahu feekum for your kind words and support. Pray for us so that Allah may accept our efforts. Visit gtaf.org to check our other apps and share with others in sha Allah!
5. Distance: 0.3807 → Reply: BarakAllahu feekum for your kind words and support. Pray for us so that Allah may accept our efforts. Visit gtaf.org to check our other apps and share with others in sha Allah!


In [ ]:
query = "Bad App"
top_matches = retrieve_reply(query, index, df, top_k=5)
for i, match in enumerate(top_matches, 1):
    print(f"{i}. Distance: {match['distance']:.4f} → Reply: {match['reply']}")

1. Distance: 0.3389 → Reply: Assalamu 'alaykum dear user. We would love to make your experience better in sha Allah. If you are still facing the same issues or have any feature requests, please provide your feedback on feedback.gtaf.org. JazakAllahu khayran.
2. Distance: 0.3389 → Reply: Assalamu 'alaykum dear user. We would love to make your experience better in sha Allah. If you are still facing the same issues or have any feature requests, please provide your feedback on feedback.gtaf.org. JazakAllahu khayran.
3. Distance: 0.3389 → Reply: Assalamu 'alaykum dear user. We would love to make your experience better in sha Allah. If you are still facing the same issues or have any feature requests, please provide your feedback on feedback.gtaf.org. JazakAllahu khayran.
4. Distance: 0.3460 → Reply: Assalamu 'alaykum dear user. We would love to make your experience better in sha Allah. If you are still facing the same issues or have any feature requests, please provide your feedback on feed